# 파생변수 만들기_Feature Engineering

- RMSE : 23724.4207

In [ ]:
concat_select['세대당_주차대수'] = concat_select['주차대수'] / (concat_select['k-전체세대수'] + 1)

In [ ]:
concat_select['계약년'] = concat_select['계약년월'].astype('str').map(lambda x : x[:4])
concat_select['계약월'] = concat_select['계약년월'].astype('str').map(lambda x : x[4:])
del concat_select['계약년월']

In [ ]:
# 따라서 2009년 이후에 지어졌으면 비교적 신축이라고 판단하고, 신축 여부 변수를 제작해보도록 하겠습니다.
concat_select['신축여부'] = concat_select['건축년도'].apply(lambda x: 1 if x >= 2009 else 0)

### 외부데이터

In [ ]:
subway = pd.read_csv('/root/AI_STAGE/upstageailab-ml-competition-ml-2/1.Data/subway_feature.csv', encoding='utf-8')
bus = pd.read_csv('/root/AI_STAGE/upstageailab-ml-competition-ml-2/1.Data/bus_feature.csv', encoding='utf-8')
bike = pd.read_csv('/root/AI_STAGE/upstageailab-ml-competition-ml-2/1.Data/bike_station.csv', encoding='cp949')
green = pd.read_csv('/root/AI_STAGE/upstageailab-ml-competition-ml-2/1.Data/green_space.csv', encoding='cp949')

In [ ]:
# 지하철 데이터로 파생변수 만들기

from sklearn.neighbors import BallTree
# 1. 역이름(호선) 문자열 만들기
subway['역_호선'] = subway['역사명'] + '(' + subway['호선'] + ')'

# 2. 위도/경도 → 라디안
coords_c = np.radians(concat_select[['위도', '경도']].values)
coords_subway = np.radians(subway[['위도', '경도']].values)

# 3. BallTree 생성
tree = BallTree(coords_subway, metric='haversine')
radius_rad = 1000 / 6371000  # 1000m → 라디안

# 4. 반경 내 인덱스 검색
indices_array = tree.query_radius(coords_c, r=radius_rad)

# 5. 이름 목록 만들기
subway_combined = subway['역_호선'].values  # '홍대입구(2호선)' 형태의 np.array

subway_names_list = []
subway_count_list = []

for indices in indices_array:
    names = subway_combined[indices] if len(indices) > 0 else []
    subway_count_list.append(len(names))
    subway_names_list.append(','.join(names))

# 6. 저장
concat_select['지하철_1000m내_개수'] = subway_count_list
concat_select['지하철_1000m내_이름목록'] = subway_names_list


In [ ]:
# 지하철 데이터로 파생 변수 만들기 

from sklearn.neighbors import BallTree

# 1. '역이름(호선)' 컬럼 생성
subway['역_호선'] = subway['역사명'] + '(' + subway['호선'] + ')'

# 2. 라디안 변환
coords_c = np.radians(concat_select[['위도', '경도']].values)
coords_subway = np.radians(subway[['위도', '경도']].values)

# 3. BallTree 생성
tree = BallTree(coords_subway, metric='haversine')

# 4. 가장 가까운 역 거리 & 인덱스 조회 (k=1)
distances, indices = tree.query(coords_c, k=1)

# 거리 변환 (라디안 → km)
nearest_distances_km = distances[:, 0] * 6371
nearest_station_names = subway['역_호선'].values[indices[:, 0]]

# 5. concat_select에 저장
concat_select['지하철_최단거리_km'] = nearest_distances_km
concat_select['지하철_최단거리_역명'] = nearest_station_names


In [ ]:
# 버스 데이터로 파생변수 만들기 

from sklearn.neighbors import BallTree

# 1. 위경도 라디안으로 변환 (주의: 위도(Y), 경도(X) 순서)
coords_c= np.radians(concat_select[['위도', '경도']].values)
coords_bus = np.radians(bus[['Y좌표', 'X좌표']].values)

# 2. BallTree 생성 (Haversine 거리 계산용)
tree_bus = BallTree(coords_bus, metric='haversine')

# 3. 가장 가까운 거리와 인덱스 구하기
bus_distances, bus_indices = tree_bus.query(coords_c, k=1)
concat_select['버스_최단거리_km'] = bus_distances[:, 0] * 6371  # km로 변환

# 4. 가장 가까운 버스 정류장 이름 추가
# 정류장명이 들어있는 컬럼명 확인 (예: '정류장명')
bus_stop_names = bus.reset_index(drop=True).loc[bus_indices[:, 0], '정류소명'].values
concat_select['버스_최단거리_정류장'] = bus_stop_names

# 5. 반경 500m 내 버스 정류장 개수 구하기
radius_km = 0.5
radius_rad = radius_km / 6371  # km → 라디안

bus_in_radius = tree_bus.query_radius(coords_c, r=radius_rad)
concat_select['버스_500m내_개수'] = [len(lst) for lst in bus_in_radius]
